# Limnpieza e Ingenieria de Caracteres

In [17]:

import numpy as np
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,silhouette_samples, silhouette_score

import scipy.stats as stats

# Carga de datos 

In [18]:
direccion_B2C_transacciones = 'Datos/base_1_transaccional.txt'
transacciones = pd.read_csv(direccion_B2C_transacciones, sep="\t", encoding='utf-8')
direccion_B2C_cotizaciones = 'Datos/base_2_cotizaciones.txt'
cotizaciones = pd.read_csv(direccion_B2C_cotizaciones, sep="\t", encoding='utf-8')

# Limpieza de datos
Se van a eliminar los datos nulos y duplicados para evitar errores en la creacion del modelo, se eliminan estos porque son muchos datos respecto al total y por lo tanto no se pierde mucha informacion.

In [20]:
# Imprimir cantidad total de datos nulos y duplicados en transacciones
print("Transacciones:")
print(f"Datos nulos totales: {transacciones.isnull().sum().sum()}")
print(f"Datos duplicados totales: {transacciones.duplicated().sum()}")

# Eliminar datos nulos y duplicados en transacciones
transacciones = transacciones.dropna()
transacciones = transacciones.drop_duplicates()

# Imprimir cantidad total de datos nulos y duplicados en cotizaciones
print("\nCotizaciones:")
print(f"Datos nulos totales: {cotizaciones.isnull().sum().sum()}")
print(f"Datos duplicados totales: {cotizaciones.duplicated().sum()}")

# Eliminar datos nulos y duplicados en cotizaciones
cotizaciones = cotizaciones.dropna()
cotizaciones = cotizaciones.drop_duplicates()


Transacciones:
Datos nulos totales: 0
Datos duplicados totales: 0

Cotizaciones:
Datos nulos totales: 0
Datos duplicados totales: 2009


In [ ]:
transacciones.head(5)

,fecha,pedido,id,edad,municipio,zona,asesor,punto de venta,cluster,categoria_macro,categoria,subcategoria,producto,color,cantidad,precio,valor,alineación con portafolio estratégico
0,1971-04-30,2,2,52,EL CARMEN DE CHUCURI,SANTANDER,asesor_2,punto_venta_2,cluster_tienda_2,categoria_macro_1,categoria_2,subcategoria_2,producto_2,GRIS,1.00,32.88,32.88,2.920320
1,1971-04-30,3,3,31,VILLANUEVA,LA GUAJIRA,asesor_3,punto_venta_2,cluster_tienda_2,categoria_macro_2,categoria_3,subcategoria_3,producto_3,BEIGE,2.00,0.56,1.13,0.117504
2,1971-04-30,4,4,43,VILLANUEVA,LA GUAJIRA,asesor_4,punto_venta_2,cluster_tienda_2,categoria_macro_3,categoria_4,subcategoria_4,producto_4,No encontrado,1.00,8.38,8.38,1.251072
3,1971-04-30,5,5,31,VILLANUEVA,LA GUAJIRA,asesor_5,punto_venta_3,cluster_tienda_3,categoria_macro_2,categoria_5,subcategoria_5,producto_5,BLANCO,21.14,2.27,47.99,3.729024
4,1971-04-30,6,6,49,ARROYOHONDO,BOLÍVAR,asesor_6,punto_venta_4,cluster_tienda_2,categoria_macro_4,categoria_6,subcategoria_6,producto_6,No encontrado,1.00,9.96,9.96,1.223424


In [ ]:
cotizaciones.head(5)

,cotizacion,estado_cotizacion,id,fecha_creacion,fecha_modificacion,categoria_macro,categoria,producto,cantidad,precio,valor
0,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_9,producto_524,1.0,34.957429,34.957429
1,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_124,1.0,2.262000,2.262000
2,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_10,producto_5085,1.0,40.600000,40.600000
3,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_188,1.0,7.863143,7.863143
4,1381864.0,Cotización Ganada,93623,1972-08-05 14:36:57.273,1972-08-05 15:06:26,categoria_macro_4,categoria_10,producto_391,1.0,63.932571,63.932571


## Asegurar tipos de datos correctos (Especialmente Fechas) 

In [ ]:
transacciones['fecha'] = pd.to_datetime(transacciones['fecha'], errors='coerce')
cotizaciones['fecha_creacion'] = pd.to_datetime(cotizaciones['fecha_creacion'], errors='coerce')
cotizaciones['fecha_modificacion'] = pd.to_datetime(cotizaciones['fecha_modificacion'], errors='coerce')
# Eliminar filas donde las fechas no se pudieron convertir (si las hay)
filas_antes_transacciones = len(transacciones)
filas_antes_cotizaciones = len(cotizaciones)

transacciones.dropna(subset=['fecha'], inplace=True)
cotizaciones.dropna(subset=['fecha_creacion', 'fecha_modificacion'], inplace=True)

filas_despues_transacciones = len(transacciones)
filas_despues_cotizaciones = len(cotizaciones)

print(f"Filas eliminadas en transacciones: {filas_antes_transacciones - filas_despues_transacciones}")
print(f"Filas eliminadas en cotizaciones: {filas_antes_cotizaciones - filas_despues_cotizaciones}")



Filas eliminadas en transacciones: 0
Filas eliminadas en cotizaciones: 0


# Ingenieria de Caracteres



In [ ]:
transacciones_1 = transacciones.copy()

# 1. Características basadas en Tiempo (Transacciones)
transacciones_1['año_venta'] = transacciones_1['fecha'].dt.year
transacciones_1['mes_venta'] = transacciones_1['fecha'].dt.month
transacciones_1['dia_semana_venta'] = transacciones_1['fecha'].dt.dayofweek # Lunes=0, Domingo=6
transacciones_1['dia_mes_venta'] = transacciones_1['fecha'].dt.day
transacciones_1['semana_año_venta'] = transacciones_1['fecha'].dt.isocalendar().week.astype(int)
transacciones_1.head(5)

,fecha,pedido,id,edad,municipio,zona,asesor,punto de venta,cluster,categoria_macro,...,color,cantidad,precio,valor,alineación con portafolio estratégico,año_venta,mes_venta,dia_semana_venta,dia_mes_venta,semana_año_venta
0,1971-04-30,2,2,52,EL CARMEN DE CHUCURI,SANTANDER,asesor_2,punto_venta_2,cluster_tienda_2,categoria_macro_1,...,GRIS,1.00,32.88,32.88,2.920320,1971,4,4,30,17
1,1971-04-30,3,3,31,VILLANUEVA,LA GUAJIRA,asesor_3,punto_venta_2,cluster_tienda_2,categoria_macro_2,...,BEIGE,2.00,0.56,1.13,0.117504,1971,4,4,30,17
2,1971-04-30,4,4,43,VILLANUEVA,LA GUAJIRA,asesor_4,punto_venta_2,cluster_tienda_2,categoria_macro_3,...,No encontrado,1.00,8.38,8.38,1.251072,1971,4,4,30,17
3,1971-04-30,5,5,31,VILLANUEVA,LA GUAJIRA,asesor_5,punto_venta_3,cluster_tienda_3,categoria_macro_2,...,BLANCO,21.14,2.27,47.99,3.729024,1971,4,4,30,17
4,1971-04-30,6,6,49,ARROYOHONDO,BOLÍVAR,asesor_6,punto_venta_4,cluster_tienda_2,categoria_macro_4,...,No encontrado,1.00,9.96,9.96,1.223424,1971,4,4,30,17


# 2. Características basadas en Tiempo (Cotizaciones)


In [ ]:
cotizaciones_1 = cotizaciones.copy()
cotizaciones_1['año_cot'] = cotizaciones_1['fecha_creacion'].dt.year
cotizaciones_1['mes_cot'] = cotizaciones_1['fecha_creacion'].dt.month
cotizaciones_1['dia_semana_cot'] = cotizaciones_1['fecha_creacion'].dt.dayofweek
cotizaciones_1['hora_cot'] = cotizaciones_1['fecha_creacion'].dt.hour
# Tiempo entre creación y modificación (puede indicar negociación o indecisión)
cotizaciones_1['dias_modificacion_cot'] = (cotizaciones_1['fecha_modificacion'] - cotizaciones_1['fecha_creacion']).dt.days
cotizaciones_1.head()


,cotizacion,estado_cotizacion,id,fecha_creacion,fecha_modificacion,categoria_macro,categoria,producto,cantidad,precio,valor,año_cot,mes_cot,dia_semana_cot,hora_cot,dias_modificacion_cot
0,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_9,producto_524,1.0,34.957429,34.957429,1972,8,5,12,0
1,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_124,1.0,2.262000,2.262000,1972,8,5,12,0
2,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_10,producto_5085,1.0,40.600000,40.600000,1972,8,5,12,0
3,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_188,1.0,7.863143,7.863143,1972,8,5,12,0
4,1381864.0,Cotización Ganada,93623,1972-08-05 14:36:57.273,1972-08-05 15:06:26,categoria_macro_4,categoria_10,producto_391,1.0,63.932571,63.932571,1972,8,5,14,0


# 3. Características Agregadas por Producto (Transacciones)


In [ ]:
transacciones_2 = transacciones_1.copy()
product_agg_trans = transacciones_2.groupby('producto').agg(
    total_unidades_vendidas=('cantidad', 'sum'),
    valor_total_ventas=('valor', 'sum'),
    precio_promedio_venta=('precio', 'mean'),
    n_transacciones_producto=('pedido', 'nunique'), # Cuántos pedidos distintos incluyen este producto
    n_clientes_producto=('id', 'nunique'),         # Cuántos clientes distintos han comprado este producto
    primera_venta=('fecha', 'min'),
    ultima_venta=('fecha', 'max')
).reset_index()

# Calcular días activo (desde primera a última venta)
product_agg_trans['dias_activo_venta'] = (product_agg_trans['ultima_venta'] - product_agg_trans['primera_venta']).dt.days + 1 # Sumar 1 para incluir el día inicial
# Frecuencia de venta promedio (transacciones por día activo)
product_agg_trans['frecuencia_venta_prod'] = product_agg_trans['n_transacciones_producto'] / product_agg_trans['dias_activo_venta']
product_agg_trans['frecuencia_venta_prod'] = product_agg_trans['frecuencia_venta_prod'].fillna(0) # Si solo hay una venta, dias_activo es 1, n_transacciones es 1. OK. Si no hay ventas no aparece.

# Unir estas características a la tabla de transacciones original
transacciones_2 = pd.merge(transacciones_2, product_agg_trans[['producto', 'total_unidades_vendidas', 'valor_total_ventas', 'precio_promedio_venta', 'n_transacciones_producto', 'n_clientes_producto','frecuencia_venta_prod']], on='producto', how='left')
transacciones_2.head(5)


,fecha,pedido,id,edad,municipio,zona,asesor,punto de venta,cluster,categoria_macro,...,mes_venta,dia_semana_venta,dia_mes_venta,semana_año_venta,total_unidades_vendidas,valor_total_ventas,precio_promedio_venta,n_transacciones_producto,n_clientes_producto,frecuencia_venta_prod
0,1971-04-30,2,2,52,EL CARMEN DE CHUCURI,SANTANDER,asesor_2,punto_venta_2,cluster_tienda_2,categoria_macro_1,...,4,4,30,17,1129.00,36537.26,32.607227,693,602,0.910644
1,1971-04-30,3,3,31,VILLANUEVA,LA GUAJIRA,asesor_3,punto_venta_2,cluster_tienda_2,categoria_macro_2,...,4,4,30,17,58336.00,33857.69,0.579439,23448,21518,30.812089
2,1971-04-30,4,4,43,VILLANUEVA,LA GUAJIRA,asesor_4,punto_venta_2,cluster_tienda_2,categoria_macro_3,...,4,4,30,17,954.00,7634.08,8.377277,786,774,1.039683
3,1971-04-30,5,5,31,VILLANUEVA,LA GUAJIRA,asesor_5,punto_venta_3,cluster_tienda_3,categoria_macro_2,...,4,4,30,17,284321.92,620946.69,2.261180,9465,7353,12.437582
4,1971-04-30,6,6,49,ARROYOHONDO,BOLÍVAR,asesor_6,punto_venta_4,cluster_tienda_2,categoria_macro_4,...,4,4,30,17,1526.00,16080.67,10.562772,1277,1245,1.680263


# 4. Características Agregadas por Producto (Cotizaciones)


In [ ]:
cotizaciones_2 = cotizaciones_1.copy()
print("\n--- Creando características agregadas por producto (Cotizaciones) ---")
product_agg_cot = cotizaciones_2.groupby('producto').agg(
    total_unidades_cotizadas=('cantidad', 'sum'),
    valor_total_cotizado=('valor', 'sum'),
    n_cotizaciones_producto=('cotizacion', 'nunique'),
    n_clientes_cotizaron=('id', 'nunique'),
    n_cotizaciones_ganadas=('estado_cotizacion', lambda x: (x == 'Cotización Ganada').sum()), # Contar ganadas
    primera_cotizacion=('fecha_creacion', 'min'),
    ultima_cotizacion=('fecha_creacion', 'max')
).reset_index()

# Tasa de conversión de cotizaciones por producto (aproximada, basada en estado)
product_agg_cot['tasa_conversion_cot_prod'] = (product_agg_cot['n_cotizaciones_ganadas'] / product_agg_cot['n_cotizaciones_producto']).fillna(0)

# Unir a la tabla de cotizaciones
cotizaciones_2 = pd.merge(cotizaciones_2, product_agg_cot[['producto', 'total_unidades_cotizadas', 'valor_total_cotizado', 'n_cotizaciones_producto', 'n_clientes_cotizaron', 'tasa_conversion_cot_prod']], on='producto', how='left')
cotizaciones_2.head(5)



--- Creando características agregadas por producto (Cotizaciones) ---


,cotizacion,estado_cotizacion,id,fecha_creacion,fecha_modificacion,categoria_macro,categoria,producto,cantidad,precio,...,año_cot,mes_cot,dia_semana_cot,hora_cot,dias_modificacion_cot,total_unidades_cotizadas,valor_total_cotizado,n_cotizaciones_producto,n_clientes_cotizaron,tasa_conversion_cot_prod
0,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_9,producto_524,1.0,34.957429,...,1972,8,5,12,0,131.0,4602.043186,88,78,0.647727
1,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_124,1.0,2.262000,...,1972,8,5,12,0,3113.0,7071.418815,1768,1585,0.802036
2,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_10,producto_5085,1.0,40.600000,...,1972,8,5,12,0,12.0,490.141515,12,9,0.750000
3,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_188,1.0,7.863143,...,1972,8,5,12,0,4635.0,36578.698818,2768,2541,0.803829
4,1381864.0,Cotización Ganada,93623,1972-08-05 14:36:57.273,1972-08-05 15:06:26,categoria_macro_4,categoria_10,producto_391,1.0,63.932571,...,1972,8,5,14,0,349.0,22408.407683,294,269,0.785714


# 5. Características Agregadas por Cliente (Transacciones) - RFM +


In [ ]:
# Estas son centradas en el cliente, pero describen el contexto de compra de un producto
transacciones_3 = transacciones_2.copy()
# Calcular la fecha de referencia (un día después de la última transacción)
fecha_ref = transacciones['fecha'].max() + pd.Timedelta(days=1)

customer_agg = transacciones_3.groupby('id').agg(
    fecha_ultima_compra=('fecha', 'max'),
    fecha_primera_compra=('fecha', 'min'),
    n_pedidos_cliente=('pedido', 'nunique'),
    n_items_distintos_cliente=('producto', 'nunique'),
    n_categorias_distintas_cliente=('categoria', 'nunique'),
    gasto_total_cliente=('valor', 'sum'),
    gasto_promedio_pedido_cliente=('valor', 'sum') # Se dividirá por n_pedidos_cliente
).reset_index()

# Recency (R)
customer_agg['recency'] = (fecha_ref - customer_agg['fecha_ultima_compra']).dt.days
# Monetary (M)
customer_agg['monetary'] = customer_agg['gasto_total_cliente']
# Valor promedio por pedido
customer_agg['gasto_promedio_pedido_cliente'] = customer_agg['gasto_total_cliente'] / customer_agg['n_pedidos_cliente']
# Tiempo de vida del cliente en días
customer_agg['tiempo_vida_cliente'] = (customer_agg['fecha_ultima_compra'] - customer_agg['fecha_primera_compra']).dt.days + 1
# Frecuencia de compra promedio (pedidos por día de vida)
customer_agg['frecuencia_compra_cliente'] = customer_agg['n_pedidos_cliente'] / customer_agg['tiempo_vida_cliente']
customer_agg['frecuencia_compra_cliente'] = customer_agg['frecuencia_compra_cliente'].fillna(0).replace(np.inf, 0)


# Unir estas características de cliente a la tabla de transacciones
transacciones_3 = pd.merge(transacciones_3, customer_agg[['id', 'recency', 'n_pedidos_cliente', 'monetary', 'n_items_distintos_cliente', 'n_categorias_distintas_cliente', 'gasto_promedio_pedido_cliente', 'frecuencia_compra_cliente']], on='id', how='left')

transacciones_3.head(5)

,fecha,pedido,id,edad,municipio,zona,asesor,punto de venta,cluster,categoria_macro,...,n_transacciones_producto,n_clientes_producto,frecuencia_venta_prod,recency,n_pedidos_cliente,monetary,n_items_distintos_cliente,n_categorias_distintas_cliente,gasto_promedio_pedido_cliente,frecuencia_compra_cliente
0,1971-04-30,2,2,52,EL CARMEN DE CHUCURI,SANTANDER,asesor_2,punto_venta_2,cluster_tienda_2,categoria_macro_1,...,693,602,0.910644,326,12,389.13,13,5,32.427500,0.027972
1,1971-04-30,3,3,31,VILLANUEVA,LA GUAJIRA,asesor_3,punto_venta_2,cluster_tienda_2,categoria_macro_2,...,23448,21518,30.812089,643,1,1.13,1,1,1.130000,1.000000
2,1971-04-30,4,4,43,VILLANUEVA,LA GUAJIRA,asesor_4,punto_venta_2,cluster_tienda_2,categoria_macro_3,...,786,774,1.039683,643,1,8.38,1,1,8.380000,1.000000
3,1971-04-30,5,5,31,VILLANUEVA,LA GUAJIRA,asesor_5,punto_venta_3,cluster_tienda_3,categoria_macro_2,...,9465,7353,12.437582,575,3,73.50,5,4,24.500000,0.038961
4,1971-04-30,6,6,49,ARROYOHONDO,BOLÍVAR,asesor_6,punto_venta_4,cluster_tienda_2,categoria_macro_4,...,1277,1245,1.680263,91,19,940.49,16,7,49.499474,0.029874


# 6. Características Agregadas por Categoría (Transacciones)


In [ ]:
transacciones_4 = transacciones_3.copy()
# Agrupar por categoría y calcular estadísticas
category_agg_trans = transacciones_4.groupby('categoria').agg(
    total_unidades_vendidas_cat=('cantidad', 'sum'),
    valor_total_ventas_cat=('valor', 'sum'),
    n_transacciones_cat=('pedido', 'nunique'),
    n_clientes_cat=('id', 'nunique'),
    n_productos_unicos_cat=('producto', 'nunique')
).reset_index()

# Unir a transacciones
transacciones_4 = pd.merge(transacciones_4, category_agg_trans, on='categoria', how='left', suffixes=('', '_cat'))
transacciones_4.head(5)



,fecha,pedido,id,edad,municipio,zona,asesor,punto de venta,cluster,categoria_macro,...,monetary,n_items_distintos_cliente,n_categorias_distintas_cliente,gasto_promedio_pedido_cliente,frecuencia_compra_cliente,total_unidades_vendidas_cat,valor_total_ventas_cat,n_transacciones_cat,n_clientes_cat,n_productos_unicos_cat
0,1971-04-30,2,2,52,EL CARMEN DE CHUCURI,SANTANDER,asesor_2,punto_venta_2,cluster_tienda_2,categoria_macro_1,...,389.13,13,5,32.427500,0.027972,291201.50,636212.34,34036,25877,86
1,1971-04-30,3,3,31,VILLANUEVA,LA GUAJIRA,asesor_3,punto_venta_2,cluster_tienda_2,categoria_macro_2,...,1.13,1,1,1.130000,1.000000,53021437.00,7784897.30,280686,173251,93
2,1971-04-30,4,4,43,VILLANUEVA,LA GUAJIRA,asesor_4,punto_venta_2,cluster_tienda_2,categoria_macro_3,...,8.38,1,1,8.380000,1.000000,115089.00,881868.25,27271,23506,175
3,1971-04-30,5,5,31,VILLANUEVA,LA GUAJIRA,asesor_5,punto_venta_3,cluster_tienda_3,categoria_macro_2,...,73.50,5,4,24.500000,0.038961,3940248.37,12652213.83,238206,146629,426
4,1971-04-30,6,6,49,ARROYOHONDO,BOLÍVAR,asesor_6,punto_venta_4,cluster_tienda_2,categoria_macro_4,...,940.49,16,7,49.499474,0.029874,107179.00,997338.62,30515,25309,298


# 7. Características de Interacción Transacción/Cotización


In [ ]:
# Esto requiere identificar si un producto cotizado fue comprado después por el mismo cliente.
# Puede ser útil para entender qué cotizaciones llevan a ventas.

# Idea: Para cada cotización de un cliente, buscar transacciones del mismo cliente posteriores a la fecha de cotización.
# Luego, verificar si alguno de los productos cotizados aparece en esas transacciones.
cotizaciones_3 = cotizaciones_2.copy()

# Ejemplo simple: Marcar si un producto cotizado fue *alguna vez* comprado por el mismo cliente
# Primero, obtener el conjunto de (cliente, producto) comprados
compras_cliente_producto = transacciones_4[['id', 'producto']].drop_duplicates()
compras_cliente_producto['comprado'] = 1

# Unir esta información a las cotizaciones
cotizaciones_merged = pd.merge(cotizaciones_3, compras_cliente_producto, on=['id', 'producto'], how='left')
cotizaciones_merged['comprado'] = cotizaciones_merged['comprado'].fillna(0).astype(int)
cotizaciones_3['producto_fue_comprado_por_cliente'] = cotizaciones_merged['comprado']

print("Cotizaciones con marca de si el producto fue comprado por ese cliente:")

cotizaciones_3.head()

Cotizaciones con marca de si el producto fue comprado por ese cliente:


,cotizacion,estado_cotizacion,id,fecha_creacion,fecha_modificacion,categoria_macro,categoria,producto,cantidad,precio,...,mes_cot,dia_semana_cot,hora_cot,dias_modificacion_cot,total_unidades_cotizadas,valor_total_cotizado,n_cotizaciones_producto,n_clientes_cotizaron,tasa_conversion_cot_prod,producto_fue_comprado_por_cliente
0,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_9,producto_524,1.0,34.957429,...,8,5,12,0,131.0,4602.043186,88,78,0.647727,1
1,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_124,1.0,2.262000,...,8,5,12,0,3113.0,7071.418815,1768,1585,0.802036,1
2,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_10,producto_5085,1.0,40.600000,...,8,5,12,0,12.0,490.141515,12,9,0.750000,1
3,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_188,1.0,7.863143,...,8,5,12,0,4635.0,36578.698818,2768,2541,0.803829,1
4,1381864.0,Cotización Ganada,93623,1972-08-05 14:36:57.273,1972-08-05 15:06:26,categoria_macro_4,categoria_10,producto_391,1.0,63.932571,...,8,5,14,0,349.0,22408.407683,294,269,0.785714,1


# 8. Características de Popularidad Relativa


In [ ]:
# Popularidad del producto dentro de su categoría
# Se necesita el total de unidades/valor por categoría (ya calculado en category_agg_trans)
transacciones_5 = transacciones_4.copy()

transacciones_5['popularidad_valor_prod_en_cat'] = transacciones_5['valor_total_ventas'] / transacciones_5['valor_total_ventas_cat']
transacciones_5['popularidad_unidad_prod_en_cat'] = transacciones_5['total_unidades_vendidas'] / transacciones_5['total_unidades_vendidas_cat']
transacciones_5.fillna({'popularidad_valor_prod_en_cat': 0, 'popularidad_unidad_prod_en_cat': 0}, inplace=True) # Manejar división por cero si una categoría no tiene ventas

# Popularidad del producto a nivel global (respecto a todas las ventas)
valor_total_global = transacciones_5['valor'].sum()
unidades_totales_global = transacciones_5['cantidad'].sum()
transacciones_5['popularidad_valor_prod_global'] = transacciones_5['valor_total_ventas'] / valor_total_global
transacciones_5['popularidad_unidad_prod_global'] = transacciones_5['total_unidades_vendidas'] / unidades_totales_global

transacciones_5.sample(5)


,fecha,pedido,id,edad,municipio,zona,asesor,punto de venta,cluster,categoria_macro,...,frecuencia_compra_cliente,total_unidades_vendidas_cat,valor_total_ventas_cat,n_transacciones_cat,n_clientes_cat,n_productos_unicos_cat,popularidad_valor_prod_en_cat,popularidad_unidad_prod_en_cat,popularidad_valor_prod_global,popularidad_unidad_prod_global
1611355,1971-11-12,719907,347581,30,CURITI,SANTANDER,asesor_248,punto_venta_10,cluster_tienda_3,categoria_macro_2,...,1.000000,9447243.52,32785902.02,348500,200620,817,0.012115,0.015566,0.004730,0.001840
641527,1972-01-07,287193,162058,43,SUBACHOQUE,CUNDINAMARCA,asesor_361,punto_venta_4,cluster_tienda_2,categoria_macro_2,...,0.176471,3940248.37,12652213.83,238206,146629,426,0.009022,0.008885,0.001359,0.000438
396186,1971-12-09,175852,112406,49,SANTA MARIA,BOYACA,asesor_282,punto_venta_6,cluster_tienda_2,categoria_macro_2,...,1.000000,1847805.04,2481481.97,96527,65223,997,0.005860,0.001113,0.000173,0.000026
916549,1971-07-24,410542,231266,41,NATAGAIMA,TOLIMA,asesor_415,punto_venta_30,cluster_tienda_1,categoria_macro_3,...,1.000000,129366.00,333685.47,26842,22060,889,0.001499,0.004793,0.000006,0.000008
1998003,1971-09-06,889754,406342,31,CURITI,SANTANDER,asesor_220,punto_venta_42,cluster_tienda_1,categoria_macro_2,...,1.000000,53021437.00,7784897.30,280686,173251,93,0.004362,0.001062,0.000404,0.000705


# Cargar Datos Limpios y transformado con nuevas variables

In [ ]:
transacciones_6 = transacciones_5.copy()
cotizaciones_4 = cotizaciones_3.copy()

# Eliminar columnas duplicadas o intermedias si es necesario
transacciones_6 = transacciones_6.loc[:,~transacciones_6.columns.duplicated()]
# Ejemplo: eliminar columnas _cat_total ya que no se necesitan directamente
transacciones_6.drop(['valor_total_ventas_cat', 'total_unidades_vendidas_cat'], axis=1, inplace=True)

# Eliminar columnas duplicadas o intermedias si es necesario
cotizaciones_4 = cotizaciones_4.loc[:,~cotizaciones_3.columns.duplicated()]


# Puedes guardar los dataframes con las nuevas características
transacciones_6.to_csv('Datos/transacciones_con_features.csv', index=False)
cotizaciones_4.to_csv('Datos/cotizaciones_con_features.csv', index=False)

In [ ]:
print(f"Tamaño de los datos es de: {transacciones_6.shape} y {cotizaciones_4.shape}")
print("\Transacciones - Nuevas columnas:")
print(transacciones_6.columns)

print("\nCotizaciones - Nuevas columnas:")
print(cotizaciones_4.columns)

Tamaño de los datos es de: (2099287, 43) y (178378, 22)
\Transacciones - Nuevas columnas:
Index(['fecha', 'pedido', 'id', 'edad', 'municipio', 'zona', 'asesor',
       'punto de venta', 'cluster', 'categoria_macro', 'categoria',
       'subcategoria', 'producto', 'color', 'cantidad', 'precio', 'valor',
       'alineación con portafolio estratégico', 'año_venta', 'mes_venta',
       'dia_semana_venta', 'dia_mes_venta', 'semana_año_venta',
       'total_unidades_vendidas', 'valor_total_ventas',
       'precio_promedio_venta', 'n_transacciones_producto',
       'n_clientes_producto', 'frecuencia_venta_prod', 'recency',
       'n_pedidos_cliente', 'monetary', 'n_items_distintos_cliente',
       'n_categorias_distintas_cliente', 'gasto_promedio_pedido_cliente',
       'frecuencia_compra_cliente', 'n_transacciones_cat', 'n_clientes_cat',
       'n_productos_unicos_cat', 'popularidad_valor_prod_en_cat',
       'popularidad_unidad_prod_en_cat', 'popularidad_valor_prod_global',
       'populari